# First test of PatternLearner network on Fugues Data

In [1]:
import os
from time import time
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.functional as TF


from Fugues_data.loader import FuguesDataset
from ML.architecture import PatternLearner, CorrelationLoss

In [2]:
DATA_PATH = os.path.join("Fugues_data", "data_16_reduced.pkl")
TEST_SIZE = 0.1
VALIDATION_SIZE = 0.2
CNN_MODEL_NAME = 'cnn_patterns_'

MAX_EPOCH = 200
BATCH_SIZE = 10
LEARNING_RATE = 0.1
EPSILON = 0.0000001
PATTERNS_MAXSIZE = (1, 4*16, 10)
PATIENCE = 3
REFINEMENT = 3  # restart training after patience runs out with the best model, decrease lr by...
LR_FAC = 0.1    # ... the learning rate factor lr_fac: lr_new = lr_old*lr_fac
LOG_INTERVAL = 60  # seconds

MINDIV = 16 #from Fugues_data.midi_to_pkl import MINDIV

torch.manual_seed(689)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load the dataset

In [3]:
data = FuguesDataset(DATA_PATH)
print(data[:].shape)

kwargs = {'num_workers': 2, 'pin_memory': True} if torch.cuda.is_available() else {'num_workers': 0}
train, validation, test_data = random_split(data, [(1-TEST_SIZE) * (1-VALIDATION_SIZE), (1-TEST_SIZE) * VALIDATION_SIZE, TEST_SIZE])
valid_data = next(iter(DataLoader(validation, len(validation), **kwargs))).float().to(device)
train_loader = DataLoader(train, BATCH_SIZE, shuffle=True, **kwargs)
test_data = next(iter(DataLoader(test_data, len(test_data), **kwargs))).float().to(device)

torch.Size([210, 1, 7712, 60])


Each music file is nearly equivalent to a picture of size 1000x500.

In [4]:
print(next(iter(train_loader)).shape)

torch.Size([10, 1, 7712, 60])


## Load architecture

In [5]:
model = PatternLearner(data[0].shape, PATTERNS_MAXSIZE)

test = torch.rand((2, *data[0].shape))
print(model(test, True).shape)
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Shape after conv 1 : torch.Size([2, 3, 964, 60])
Shape after conv 2 : torch.Size([2, 6, 120, 60])
Shape after conv 3 : torch.Size([2, 12, 15, 60])
Shape after conv 4 : torch.Size([2, 24, 1, 6])
Shape after dense Layer : torch.Size([2, 640])
torch.Size([2, 1, 64, 10])
Number of parameters: 177137


## Train functions

In [6]:
OPTIMIZER = Adam
LOSS_FUNCTION = CorrelationLoss().minmax_regul(beta=0.1)

In [ ]:
def train_epoch_cnn(model, optimizer):
    """
    Training loop for one epoch of NN training.
    """
    model.train()  # set model to training mode (activate dropout layers if any)
    t = time() # we measure the needed time
    for batch_idx, input_data in enumerate(train_loader):  # iterate over training input_data
        input_data = input_data.float().to(device)  # move input_data to device (GPU) if necessary
        optimizer.zero_grad()  # reset optimizer
        output = model(input_data)   # forward pass: calculate output of network for input_data
        loss = LOSS_FUNCTION(output, input_data)

        loss.backward()  # backward pass: calculate gradients using automatic diff. and backprop.
        optimizer.step()  # udpate parameters of network using our optimizer
        cur_time = time()
        # print some outputs if we reached our logging interval
        if cur_time - t > LOG_INTERVAL or batch_idx == len(train_loader)-1:  
            print(f"[{batch_idx * BATCH_SIZE}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]",
                  f"\tloss: {loss.item():.6f}, took {cur_time - t:.2f}s")
            t = cur_time


def valid_cnn(model):
    """
    Test loss evaluation
    """
    model.eval()  # set model to inference mode (deactivate dropout layers)
    with torch.no_grad():  # do not calculate gradients since we do not want to do updates
        output = model(test_data)
        loss = LOSS_FUNCTION(output, test_data)
    print(f'Average eval loss: {loss:.4f}\n')
    return loss

In [8]:
def train_cnn():
    """
    Run CNN training using the datasets.

    Return
    ------
        nn.Model
            trained model
    """

    # create model and optimizer, we use plain SGD with momentum
    optimizer = OPTIMIZER(model.parameters(), lr=LEARNING_RATE)

    model_cnt = 0
    new_model_file = os.path.join(CNN_MODEL_NAME + str(model_cnt) + '.model')
    while os.path.exists(new_model_file):
        model_cnt += 1
        new_model_file = os.path.join(CNN_MODEL_NAME + str(model_cnt) + '.model')
    

    # train model for max_epochs epochs, output loss after log_intervall seconds.
    # for each epoch run once on validation set, 
    # write model to disk if validation loss decreased
    # if validation loss increased, check for early stopping with patience and refinements
    # after model is trained, perform a run on test set and output loss (don't forget to reload best model!)
    best_valid_loss = 9999.
    cur_patience = PATIENCE
    cur_refin = REFINEMENT

    #model.load_state_dict(torch.load(last_model_file, map_location=device).state_dict())
    print('Training CNN...')
    start_t = time()

    for epoch in range(1, MAX_EPOCH+1):
        train_epoch_cnn(model, optimizer)
        valid_loss = valid_cnn(model)

        if valid_loss < best_valid_loss:
            torch.save(model, new_model_file)
            best_valid_loss = valid_loss
            cur_patience = PATIENCE

        elif cur_patience <=0:
            model.load_state_dict(torch.load(new_model_file, map_location=device).state_dict())
            if cur_refin <= 0:
                print("Max refinement reached !")
                break
            else:
                print("Max patience reached !")
                
                cur_patience = PATIENCE
                for param_group in optimizer.param_groups:
                    lr = LR_FAC * param_group['lr']
                    param_group['lr'] = lr
                cur_refin -= 1
            
        else:
            print("We still have patience...")
            cur_patience -= 1
    
    print(f'Training took: {time()-start_t:.2f}s for {epoch} epochs')
    
    return model



def load_cnn(load_model:str):
    "Load the model."
    if load_model is None or not os.path.exists(load_model):
        print('Model file not found, unable to load...')
    else:
        model.load_state_dict(torch.load(load_model, map_location=device).state_dict())
        print("Model file loaded: {}".format(load_model))
    return model

In [9]:
train_cnn()

Training CNN...
[50/152 (31%)] 	loss: -0.220237, took 67.17s
[110/152 (69%)] 	loss: -0.197817, took 62.34s
[30/152 (94%)] 	loss: -0.165877, took 33.72s
Average eval loss: -0.2128

[50/152 (31%)] 	loss: -0.215598, took 63.18s
[110/152 (69%)] 	loss: -0.236806, took 62.11s
[30/152 (94%)] 	loss: -0.200092, took 37.48s
Average eval loss: -0.2101

We still have patience...


KeyboardInterrupt: 